In [3]:
#default_exp dr

In [4]:
#hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# Dimensionality reduction

In [6]:
#export
import umap
import cudf
import cuml
import pandas as pd
import numpy as np
from fastcore.all import *
from dvats.imports import *
from dvats.load import TSArtifact

In [7]:
#export
def check_compatibility(dr_ar:TSArtifact, enc_ar:TSArtifact):
    "Function to check that the artifact used by the encoder model and the artifact that is \
    going to be passed through the DR are compatible"
    try:
        # Check that both artifacts have the same variables
        chk_vars = dr_ar.metadata['TS']['vars'] == enc_ar.metadata['TS']['vars']
        # Check that both artifacts have the same freq
        chk_freq = dr_ar.metadata['TS']['freq'] == enc_ar.metadata['TS']['freq']
        # Check that the dr artifact is not normalized (not normalized data has not the key normalization)
        chk_norm = dr_ar.metadata['TS'].get('normalization') is None
        # Check that the dr artifact has not missing values
        chk_miss = dr_ar.metadata['TS']['has_missing_values'] == "False"
        # Check all logical vars.
        if chk_vars and chk_freq and chk_norm and chk_miss:
            print("Artifacts are compatible.")
        else:
            raise Exception
    except Exception as e:
        print("Artifacts are not compatible.")
        raise e
    return None


In [8]:
#export
import warnings
from numba.core.errors import NumbaPerformanceWarning
@delegates(cuml.UMAP)
def get_UMAP_prjs(input_data, cpu=True, **kwargs):
    "Compute the projections of `input_data` using UMAP, with a configuration contained in `**kwargs`."
    warnings.filterwarnings("ignore", category=NumbaPerformanceWarning) # silence NumbaPerformanceWarning
    reducer = umap.UMAP(**kwargs) if cpu else cuml.UMAP(**kwargs)
    projections = reducer.fit_transform(input_data)
    return projections

In [9]:
foo = np.random.rand(5, 10)
bar = get_UMAP_prjs(foo, cpu=True, n_neighbors=3, min_dist=0.1)
test_eq(bar.shape, (foo.shape[0], 2))

If you want to have consistent results across executions, use `random_state`

In [10]:
bar = get_UMAP_prjs(foo, cpu=True, n_neighbors=3, random_state=1234)
baz = get_UMAP_prjs(foo, cpu=True, n_neighbors=3, random_state=1234)
test_eq(bar, baz)

## Export - 

In [12]:
#hide
from nbdev.export import notebook2script
notebook2script()
beep(1)

Converted 02a_encoder_DCAE-torch.ipynb.
Converted dr.ipynb.
Converted encoder.ipynb.
Converted index.ipynb.
Converted load.ipynb.
Converted utils.ipynb.
Converted visualization.ipynb.
